<a href="https://colab.research.google.com/github/johanhoffman/DD2363-VT19/blob/bozzato/Lab-3/bozzato_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 3: Iterative methods**
**Bozzato Federico**

# **Abstract**

Short summary of the lab report. State the objectives, methods used, main results and conlusions. 

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [1]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2019 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them. 

In [0]:
# Load neccessary modules.
from google.colab import files

import time
import sys
import numpy as np
from numpy import linalg


# **Introduction**

Give a short description of the problem investigated in the report, and provide some background information so that the reader can understand the context. 

Briefly describe what method you have chosen to solve the problem, and justify why you selected that method. 

Here you can express mathematics through Latex syntax, and use hyperlinks for references.

[Hyperlink to DD2363 course website.](https://kth.instructure.com/courses/7500)

$
{\displaystyle \frac{\partial u}{\partial t}} + u\cdot \nabla u +\nabla p = f, \quad \nabla \cdot u=0$



# **Methods**

## Mandatory assignment

### Jacobi iteration

In [0]:
# REFERENCE: https://en.wikipedia.org/wiki/Jacobi_method



def jacobiIteration(matrixA ,b, tol=1e-5):
  
  # split matrixA= D + E= D + A-D
  if not isinstance(matrixA,np.ndarray):
    matrixA= np.array(matrixA)
    
  n= matrixA.shape[0]
  
  
  # TODO: assure A1 does not have 0 in the diagonal. If yes, permutation
  
  
  
  D= np.diag(matrixA)
  invD= np.diag([1/D[i] for i in range(0,matrixA.shape[0])])
  E= matrixA - np.diag(D)
  
  
  # calculating M and c
  M= -np.dot(invD,E)
  c= np.dot(invD,b)
  
  # convergence if rho(M)<1
  eigvals,_= np.linalg.eig(M)
  print(eigvals)
  if max(abs(eigvals)) >= 1:
    print('Convergence will not be reached because rho(I-invD*A)>=1.\nTry other methods.')
    sys.exit(1)

  x= np.zeros((n,1))
  res = b - np.dot(matrixA,x)    # initial residual -> just to enter in the loop
  res= linalg.norm(res)
  norm_b= linalg.norm(b)
  it= 1
  while res/norm_b > tol:
    x= np.dot(M,x) + c
    
    res = b - np.dot(matrixA,x)    
    res= linalg.norm(res)
    it+= 1
  
  print('solution after {} iterations'.format(it))
  return x

In [4]:
#TODO: moving this cell to Result section and add examples
A= [[1,  0],
    [1, -1]]

b= [[11],
    [13]]

x= jacobiIteration(A,b,1e-10)
print('x.T=',x.T)

[-0. -0.]
solution after 3 iterations
x.T= [[11. -2.]]


### Gauss-Seidel iteration

In [0]:
def gaussSeidelIteration(matrixA ,b, tol=1e-5):
  if not isinstance(matrixA,np.ndarray):
    matrixA= np.array(matrixA)
  
  n= matrixA.shape[0]
  
  # split matrixA= L + U
  L= np.tril(matrixA)
  U= np.triu(matrixA,1)
    
  print('U=\n{}'.format(U))
  print('L=\n{}'.format(L))
    
  invL= linalg.inv(L)
  print('invL=\n{}'.format(invL))
  M= -np.dot(invL,U)
  c= np.dot(invL,b)
  print('M=\n{}'.format(M))
  
  eigvals,_= np.linalg.eig(M)
  print('Eig=',eigvals)
  if max(abs(eigvals)) >= 1:
    print('Convergence will not be reached because rho(I-invD*A)>=1.\nTry other methods.')
    sys.exit(1)
    
  x= np.zeros((n,1))
  res = b - np.dot(matrixA,x)    # initial residual -> just to enter in the loop
  res= linalg.norm(res)
  norm_b= linalg.norm(b)
  it= 1
  while res/norm_b > tol:
    x= np.dot(M,x) + c
    
    res = b - np.dot(matrixA,x)    
    res= linalg.norm(res)
    it+= 1
  
  print('solution after {} iterations'.format(it))
  return x
  

In [6]:
A= [[16,  3],
    [7, -11]]

b= [[11],
    [13]]

gaussSeidelIteration(A,b)

U=
[[0 3]
 [0 0]]
L=
[[ 16   0]
 [  7 -11]]
invL=
[[ 0.0625      0.        ]
 [ 0.03977273 -0.09090909]]
M=
[[-0.         -0.1875    ]
 [-0.         -0.11931818]]
Eig= [-0.         -0.11931818]
solution after 7 iterations


array([[ 0.81218576],
       [-0.6649727 ]])

## Extra assignment

### GMRES method

In [0]:
def GMRESiteration(matrixA, b, tol=1e-5):
  
  ######################################
  ##             ARNOLDI              ##
  ######################################
  def arnoldiIteration(matrixA, Q, K):
    if not isinstance(matrixA, np.ndarray):
      matrixA= np.array(matrixA)
    
    if not isinstance(Q, np.ndarray):
      Q= np.array(Q)

    vect_h= np.zeros((K+1,1))
    q= np.dot(matrixA, Q[:,K-1])  
    
    for k in range(0,K):
      vect_h[k]= np.inner(Q[:,k],q)
      q= q - vect_h[k]*Q[:,k]
      
    vect_h[K]= linalg.norm(q)
    q= q / vect_h[K]

    return vect_h, q
  
  ######################################
  ##              GMRES               ##
  ######################################
  if not isinstance(matrixA,np.ndarray):
    matrixA= np.array(matrixA)
    
  if not isinstance(b,np.ndarray):
    b= np.array(b)
  
  n= matrixA.shape[0]
  k= 1
  Q= np.zeros((n,1))
  Q[:]= b / linalg.norm(b)
  #while res / res_0 >= tol:
  while k < 5:
    h_new, q_new= arnoldiIteration(matrixA, Q, k)
    
    # update Q
    Q_new= np.zeros((n,k+1))
    Q_new[:,0:k]= Q
    Q_new[:,k]= q_new
    Q= Q_new
    
    # update H
    if k == 1:
      H= h_new
    else:
      H_new= np.zeros((k+1,k))
      H_new[0:k,0:k-1]= H
      #print(H_new)
      #print(H_new.shape)
      #print(h_new)
      H_new[:,k-1]= h_new.T
      H= H_new
      
    print('------------------------------')
    print('Q=\n{}'.format(Q))
    print('H=\n{}'.format(H))
    print('------------------------------')
    
    k+= 1

In [84]:
A= [[16,  3],
    [7, -11]]

b= [[11],
    [13]]

GMRESiteration(A,b)

------------------------------
Q=
[[ 0.64594224  0.76338629]
 [ 0.76338629 -0.64594224]]
H=
[[ 5.19655172]
 [12.14137931]]
------------------------------
------------------------------
Q=
[[ 0.64594224  0.76338629  0.64556387]
 [ 0.76338629 -0.64594224  0.76370628]]
H=
[[ 5.19655172e+00  1.61413793e+01]
 [ 1.21413793e+01 -1.96551724e-01]
 [ 0.00000000e+00  6.57812998e-15]]
------------------------------
------------------------------
Q=
[[ 0.64594224  0.76338629  0.64556387  0.76370628]
 [ 0.76338629 -0.64594224  0.76370628 -0.64556387]]
H=
[[ 5.19655172e+00  1.61413793e+01  5.18855239e+00]
 [ 1.21413793e+01 -1.96551724e-01  1.21414752e+01]
 [ 0.00000000e+00  6.57812998e-15  1.82505924e-15]
 [ 0.00000000e+00  0.00000000e+00  7.83238976e-16]]
------------------------------
------------------------------
Q=
[[ 0.64594224  0.76338629  0.64556387  0.76370628 -0.64513406]
 [ 0.76338629 -0.64594224  0.76370628 -0.64556387 -0.76406939]]
H=
[[ 5.19655172e+00  1.61413793e+01  5.18855239e+00  1.

# **Results**

Present the results. If the result is an algorithm that you have described under the *Methods* section, you can present the data from verification and performance tests in this section. If the result is the output from a computational experiment this is where you present a selection of that data. 

# **Discussion**

Summarize your results and your conclusions. Were the results expected or surprising. Do your results have implications outside the particular problem investigated in this report? 

# References

In this appendix some examples are given to express and visualizing mathematical concepts such as vectors, matrices, meshes and functions. 

This is not part of the template report for the course.